#### Decision Variables

-   **First-stage variables**

    $$x \in \mathbb{R}^{n_1}, \quad x \geq 0$$

    Decided before uncertainty is revealed; common across all scenarios.

-   **Second-stage (recourse) variables**

    $$y(\omega) \in \mathbb{R}^{n_2}, \quad y(\omega) \geq 0 \quad \forall \omega \in \Omega$$

    Adapted to each scenario outcome.

#### Parameters

-   $c \in \mathbb{R}^{n_1}$: Cost vector for first-stage decisions.
-   $A \in \mathbb{R}^{m_1 \times n_1}$, $b \in \mathbb{R}^{m_1}$: First-stage constraint matrix and RHS.
-   For each scenario $\omega$ with probability $p_{\omega}$:
    -   $T_{\omega} \in \mathbb{R}^{m_2 \times n_1}$, $W_{\omega} \in \mathbb{R}^{m_2 \times n_2}$,
    -   $r_{\omega} \in \mathbb{R}^{m_2}$, $g_{\omega} \in \mathbb{R}^{n_2}$.


#### Objective Function

$$\min_{x \geq 0} c^\top x + \mathbb{E}_{\tilde{\omega}} [h(x, \tilde{\omega})],$$

where the second-stage recourse cost:

$$h(x, \omega) = \min_{y \geq 0} \left\{ g_\omega^\top y \mid W_\omega y \geq r_\omega - T_\omega x \right\}$$

#### Constraints

1. **First-stage feasibility**  
   $$A x \leq b$$

2. **Recourse feasibility (per scenario)**  
   $$W_\omega y(\omega) \geq r_\omega - T_\omega x, \quad \forall \omega \in \Omega$$

3. **Nonanticipativity**  
   Implicit: $x$ is the same across all scenarios (no $x(\omega)$ subscript).


### The model which is used according to the paper is:


#### First-stage:

-   $x_b$: Lumber procured (board feet)
-   $x_f$: Finishing labor hours procured
-   $x_c$: Carpentry labor hours procured

#### Second-stage:

For each scenario $\omega \in \{\text{low, mid, high}\}$:

-   $y_{d,\omega}$: Desks produced in scenario $\omega$
-   $y_{t,\omega}$: Tables produced in scenario $\omega$
-   $y_{c,\omega}$: Chairs produced in scenario $\omega$

#### Objective

Maximize expected profit:

$$\text{Minimize } 2x_b +4x_f +5.2x_c - \sum_{\omega} p_\omega(60y_{d,\omega} + 40y_{t,\omega} + 10y_{c,\omega})$$

where $p_\omega$ is the probability of scenario $\omega$.


In [3]:
import gurobipy as gp
from gurobipy import GRB

In [14]:
# according to the given values in the paper
scenarios   = ['l','m','h']
p           = {'l': 0.3, 'm': 0.4, 'h': 0.3}   # scenario probabilities
cost_x      = {'b': 2.0, 'f': 4.0, 'c': 5.2}   # first‐stage costs

# second‐stage unit profits
profit = {'d': 60.0, 't': 40.0, 'c': 10.0}

usage = {
    'b': {'d':  8.0, 't':  6.0, 'c': 1.0},
    'f': {'d':  4.0, 't':  2.0, 'c': 1.5},
    'c': {'d':  2.0, 't': 1.5, 'c': 0.5},
}

# demand (or capacity) upper bounds per scenario and product
demand_ub = {
    'l': {'d': 50,  't':  20,  'c': 200},
    'm': {'d':150,  't': 110,  'c': 225},
    'h': {'d':250,  't': 250,  'c': 500},
}

In [15]:
m = gp.Model("two_stage_furniture")

In [16]:
x = m.addVars(['b','f','c'], lb=0.0, name="x") # First‐stage vars: x_b, x_f, x_c
y = m.addVars(['d','t','c'], scenarios, lb=0.0, name="y") # Recourse vars: y[i,ω] for i∈{d,t,c}

In [17]:
# First‐stage cost + expected second‐stage profit
first_stage_cost = sum(cost_x[r] * x[r] for r in ['b','f','c'])
recourse_profit  = sum(
    p[w] * (profit['d']*y['d',w]
           + profit['t']*y['t',w]
           + profit['c']*y['c',w])
    for w in scenarios
)
m.setObjective(first_stage_cost - recourse_profit, GRB.MINIMIZE)

In [18]:
# Resource‐capacity constraints per scenario:
for w in scenarios:
    for r in ['b','f','c']:
        m.addConstr(
            - x[r]
            + sum(usage[r][i] * y[i,w] for i in ['d','t','c'])
            <= 0,
            name=f"res_{r}_{w}"
        )

# Demand/capacity upper bounds (recourse side)
for w in scenarios:
    for i in ['d','t','c']:
        m.addConstr(
            y[i,w] <= demand_ub[w][i],
            name=f"demand_{i}_{w}"
        )


In [20]:
m.write("two stage model.lp")
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 18 rows, 12 columns and 45 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 8e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+02]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.730000000e+03


In [24]:
if m.status == GRB.OPTIMAL:
    print("First‐stage x:")
    for r in ['b','f','c']:
        print(f" x_{r} = {x[r].X:.2f}")
    print("\nRecourse y (per scenario):")
    for w in scenarios:
        print(f" Scenario {w}: ",
              ", ".join(f"{i}={y[i,w].X:.0f}" for i in ['d','t','c']))
else:
    print("No optimal solution found.")


First‐stage x:
 x_b = 1300.00
 x_f = 540.00
 x_c = 325.00

Recourse y (per scenario):
 Scenario l:  d=50, t=20, c=200
 Scenario m:  d=80, t=110, c=0
 Scenario h:  d=80, t=110, c=0
